https://petrobras.com.br/pt/nossas-atividades/precos-de-venda-de-combustiveis/

In [8]:
import pandas as pd
import tabula 
from urllib import request
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin
import re
from PyPDF2 import PdfFileReader
import matplotlib.pyplot as plt
import yfinance as yf

In [9]:
url = "https://petrobras.com.br/pt/nossas-atividades/precos-de-venda-as-distribuidoras/gasolina-e-diesel/"
html = urlopen(url)
bs = BeautifulSoup(html, "html.parser")

# GASOLINA
links = []
for x in bs.find_all("a", href = True):
    if re.search(re.compile("gasolina", flags = re.IGNORECASE), str(x)):
        links.append(x["href"])

root = "https://petrobras.com.br/"

link_final_gasolina = links[3][9:]
link_gasolina = root + link_final_gasolina

pag_pdf = tabula.read_pdf(link_gasolina, pages = "all", multiple_tables=True)#, pandas_options= {"header": None})

TRATAMENTO

In [10]:
agrupado = pd.DataFrame()
for dataframe in range(0,len(pag_pdf)):
    if len(pag_pdf[dataframe]) > 70:        
        df = pag_pdf[dataframe]
        for line_num, line in enumerate(pag_pdf[dataframe].iloc[:,0]):
            if line == "LOCAL":
                linha_datas = df.iloc[line_num, :]
        df2 = df.dropna(subset = [df.columns[0]])
        df2.columns = linha_datas
        df2 = df2.reset_index(drop = True)
        df2.drop(0, axis = 0, inplace = True)
        df3 = pd.melt(df2, id_vars = df2.columns[:2], value_vars = df2.columns[2:])
        df3.columns = ["LOCAL", "MODALIDADE", "DATA", "VALOR"]
        agrupado = pd.concat([agrupado, df3], axis = 0)

In [11]:
agrupado.dropna(subset = ["VALOR"], inplace=True)

agrupado.loc[:,"VALOR"] = agrupado["VALOR"].str.replace(".", "")
agrupado.loc[:,"VALOR"] = agrupado["VALOR"].apply(lambda x: float(x.replace(",", "."))/1000)
agrupado.loc[:,"DATA"] = agrupado["DATA"].apply(lambda x: pd.to_datetime(x, format = "%d.%m.%Y"))

C:/Users/MATHEU~1.VIZ/AppData/Local/Temp/xpython_33256/2266775821.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  agrupado.loc[:,"VALOR"] = agrupado["VALOR"].str.replace(".", "")
